# WScript or CScript Dropper
Detects wscript/cscript executions of scripts located in user directories

## Rule Content
```
- title: WScript or CScript Dropper
  id: cea72823-df4d-4567-950c-0b579eaf0846
  status: experimental
  description: Detects wscript/cscript executions of scripts located in user directories
  author: Margaritis Dimitrios (idea), Florian Roth (rule)
  tags:
  - attack.defense_evasion
  - attack.execution
  - attack.t1064
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      Image:
      - '*\wscript.exe'
      - '*\cscript.exe'
      CommandLine:
      - '* C:\Users\\*.jse *'
      - '* C:\Users\\*.vbe *'
      - '* C:\Users\\*.js *'
      - '* C:\Users\\*.vba *'
      - '* C:\Users\\*.vbs *'
      - '* C:\ProgramData\\*.jse *'
      - '* C:\ProgramData\\*.vbe *'
      - '* C:\ProgramData\\*.js *'
      - '* C:\ProgramData\\*.vba *'
      - '* C:\ProgramData\\*.vbs *'
    falsepositive:
      ParentImage: '*\winzip*'
    condition: selection and not falsepositive
  fields:
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - Winzip
  - Other self-extractors
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_path.keyword:(*\\wscript.exe OR *\\cscript.exe) AND process_command_line.keyword:(*\ C\:\\Users\\*.jse\ * OR *\ C\:\\Users\\*.vbe\ * OR *\ C\:\\Users\\*.js\ * OR *\ C\:\\Users\\*.vba\ * OR *\ C\:\\Users\\*.vbs\ * OR *\ C\:\\ProgramData\\*.jse\ * OR *\ C\:\\ProgramData\\*.vbe\ * OR *\ C\:\\ProgramData\\*.js\ * OR *\ C\:\\ProgramData\\*.vba\ * OR *\ C\:\\ProgramData\\*.vbs\ *)) AND (NOT (process_parent_path.keyword:*\\winzip*)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()